## Installing libraries:

In [1]:
!pip install biopython
!pip install pandas

## Importing libraries:

In [2]:
from Bio import SeqIO
from glob import glob
from collections import Counter
import pandas as pd
import os
from itertools import chain

## Organizing all the genbank files into a list:

In [3]:
# Returns current directory 
file_path = os.getcwd()

# Search for all files in the current directory with .gb extension
file_names = glob(file_path + "/*.gb") 

# Storing all the genbank files into a list after processing through SeqIO():
gb_objs = [SeqIO.read(file_names[i], "gb") for i in range(len(file_names))]

## Searching for viral genes in all of the genbank files:

In [4]:
# Creating our text object:
vir_genes_txt = file_path + '\\vir_factors.txt'

# Storing name of each gene in a list using file reading:
with open(vir_genes_txt, 'r') as input_file:
    
    vir_genes = [gene.strip() for gene in input_file]

In [5]:
# Storing all the gene sequence objects of each genome in a 2D list:
genes_list_all = [[gene for gene in gb_objs[i].features if gene.type == 'gene'] for i in range(len(gb_objs))]

In [6]:
# Creating an empty 2D list for storing all the viral gene sequences: 
vir_gene_seq = [[] for i in range(len(gb_objs))]

# Storing all the viral gene sequences from each bacterial genome: 
for i in range(len(gb_objs)):
    
    for gene in genes_list_all[i]:

        if 'gene' in gene.qualifiers.keys():

            gene_name = gene.qualifiers['gene'][0]

            if gene_name in vir_genes:
                extract = gene.extract(gb_objs[i])
                extract.id = gene_name
                vir_gene_seq[i].append(extract)

In [29]:
# Determing the bacterial genome that has any one of the viral genes:

for i in range(len(gb_objs)):
    
    if vir_gene_seq[i] != []:
        
        vir_gene_names = []
        
        for j in range(len(vir_gene_seq[i])):
            vir_gene_names.append(vir_gene_seq[i][j].id)
            
        print((os.path.split(file_names[i])[-1]).strip(".gb") + " has the following viral genes:\n" + str(vir_gene_names), "\n")

e_coli has the following viral genes:
['ecpE', 'ecpD', 'ecpC', 'ecpB', 'ecpA', 'ecpR', 'paa', 'lpfB', 'espF', 'escF', 'cesD', 'espB', 'espD', 'espA', 'sepL', 'escD', 'eae', 'tir', 'escN', 'escV', 'escJ', 'escC', 'cesD', 'escU', 'escT', 'escS', 'ler', 'lpfA'] 

Mycobacterium tuberculosis has the following viral genes:
['espD', 'espA', 'espF', 'espB'] 



In [32]:
# Saving all the viral genes in a fasta file for each bacterial genome:

for i in range(len(vir_gene_seq)):
    
    if vir_gene_seq[i] != []:
        fasta_file = os.getcwd() + "\\vir_genes_" + (os.path.split(file_names[i])[-1]).strip(".gb") + ".fasta"
        SeqIO.write(vir_gene_seq[i], fasta_file, "fasta")
        print("Fasta file for the viral genes in " + (os.path.split(file_names[i])[-1]).strip(".gb") + " has been saved")
    else:
        print((os.path.split(file_names[i])[-1]).strip(".gb") + " genome doesn't contain any viral genes in it")

Fasta file for the viral genes in e_coli has been saved
Fasta file for the viral genes in Mycobacterium tuberculosis has been saved
Pneumoniae genome doesn't contain any viral genes in it
salmonella genome doesn't contain any viral genes in it


## Further Analysis:

### Counting the number of unique gene sequences in each genbank file:

In [35]:
features_counts = []

for i in range(len(file_names)):
    
    features_counts_each = dict(Counter([features.type for features in gb_objs[i].features])) 
    features_counts.append(features_counts_each)

print(features_counts)

[{'source': 1, 'gene': 5329, 'CDS': 5067, 'misc_feature': 160, 'rRNA': 22, 'tRNA': 103, 'repeat_region': 65, 'tmRNA': 1}, {'source': 1, 'gene': 4069, 'CDS': 3948, 'tRNA': 45, 'repeat_region': 262, 'misc_feature': 29, 'ncRNA': 22, 'mobile_element': 56, 'rRNA': 3, 'tmRNA': 1}, {'source': 1, 'gene': 2402, 'CDS': 2155, 'tRNA': 55, 'rRNA': 12, 'ncRNA': 2, 'misc_binding': 7, 'misc_feature': 8, 'tmRNA': 1}, {'source': 1, 'gene': 4649, 'CDS': 4323, 'rRNA': 22, 'tRNA': 78, 'misc_RNA': 9, 'misc_feature': 4}]


In [37]:
# Set of unique gene sequences across all the genebank files:
features_set = set(chain.from_iterable([list(features_counts[i].keys()) for i in range(len(gb_objs))]))
print(features_set)

{'gene', 'repeat_region', 'source', 'misc_RNA', 'tmRNA', 'ncRNA', 'CDS', 'mobile_element', 'misc_binding', 'misc_feature', 'tRNA', 'rRNA'}


### Creating a pandas dataframe to store the counts of each unique gene sequence:

In [39]:
# Creates an universal list of unique gene seqeunces and their counts
for i in range(len(features_counts)):
    
    for feature in features_set:
    
        if feature not in features_counts[i].keys():
            features_counts[i][feature] = 0
            
# Creating a pandas dataframe
df = pd.DataFrame(features_counts)

df['GenBank file'] = [(os.path.split(file_names[i])[-1]).strip(".gb") for i in range(len(gb_objs))]
df = df.set_index("GenBank file")

# Saving pandas dataframe as csv file:
df.to_csv(os.getcwd() + "\\GenBank files features count.csv", index = True)